In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\SAIDT.dll"
#r ".\binaries\ApplicationWithIDT.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using ApplicationWithIDT;
using SAIDT;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


# Run Simulation

## SinglePhaseField

In [3]:
string dbPath=null;

In [48]:
using SAIDT;
BoSSS.Solution.Application.InitMPI();
BoSSS.Solution.Application.DeleteOldPlotFiles();
var pSAIDT = new SAIDTMain();
var CSAIDT = SAIDTHardCodedControl.StraightShock(
                    dbPath: dbPath,
                    MaxIterations: 50,
                    dgDegree: 0,
                    numOfCellsX: 4,
                    numOfCellsY: 4,
                    OptiNumOfCellsX: 4,
                    OptiNumOfCellsY: 4,
                    agg: 0.4,
                    ImmediatePlotPeriod: -1,
                    optiLevelSetType: OptiLevelSetType.SinglePhaseField,
                    LSDegree: 3
                    );
CSAIDT.SessionName="SAIDT_DGLevelSet";
CSAIDT.ProjectName="SAIDT_DGLevelSet";
pSAIDT.Init(CSAIDT);
pSAIDT.InitializeEverything();


rm DGLevelSet.plt;
Session ID: 00000000-0000-0000-0000-000000000000, DB path: 'EMPTY'
Session directory 'EMPTY\sessions\00000000-0000-0000-0000-000000000000'.
Grid repartitioning method: METIS
Grid repartitioning options: 
Number of cell Weights: 0


*****************************************************************************************
*  X   X   DDD   GGG   -   III  SSS  TTTTT  *
*   X X    D  D G      -    I  S      T     *
*    X     D  D G  GG  -    I   SSS   T     *      XDG Implicit Shock Tracking Solver    
*   X X    D  D G   G  -    I      S  T     *
*  X   X   DDD   GGG   -   III  SSS   T     *
*****************************************************************************************
*  
*  solver:                             SAIDT.SAIDTMain
*  project:                            SAIDT_DGLevelSet
*  session:                            SAIDT_DGLevelSet
*  optimization problem:               FullEnRes
*  merit function:                     L1Merit
*  level set type:          

In [69]:
Func<double[],double> discCirc= delegate(double[] X){
    var y=X[1];
    var x=X[0];
    double result=0;
    if (y > 0.5 && x > 0.5)
        {
            // Apply the first function for y > 0.5
            result += Math.Pow(x-0.5,2) + Math.Pow(y-0.5,2) - 0.36*0.36;
        }
    else if (y <= 0.5 && x > 0.5)
    {
        // Apply the second function for y < 0.75
        result += Math.Pow(x-0.5,2) + Math.Pow(y-0.5,2)  - 0.3*0.3;
    }else if (y <= 0.5 && x <= 0.5)
    {
        // Apply the second function for y < 0.75
        result += Math.Pow(x-0.5,2) + Math.Pow(y-0.5,2)  - 0.35*0.35;
    }else if (y > 0.5 && x <= 0.5)
    {
        // Apply the second function for y < 0.75
        result += Math.Pow(x-0.5,2) + Math.Pow(y-0.5,2)  - 0.29*0.29;
    }
    
    // int i = (int)(y / 0.2);
    // double result;
    // if(x<0.6){
    //     result = (x - 0.1 * i) - 0.5 * y;

    // }else{
    //     result = (x - 0.6 * i) - 0.5 * (y);
    // }
    // Calculate the function value
    
    return result;
};

In [68]:
using BoSSS.Solution.Tecplot;
using System.IO;
Directory.GetFiles(".", "*_SP_*").ForEach(file => File.Delete(file));

public void plotSF(string name){
    List<DGField> flds= new List<DGField>();
foreach(var field in pSAIDT.RegisteredFields){
    if(field is XDGField xfield){
        flds.Add(xfield.GetSpeciesShadowField("L"));
        flds.Add(xfield.GetSpeciesShadowField("R"));
        flds.Add(field);
    }else{
        flds.Add(field);
    }
}
var texplot = new Tecplot(flds[0].GridDat, 2);
texplot.PlotFields( name,0.0,flds);
}
Func<double[],double> discCircA= delegate(double[] X){
    var y=X[1];
    var x=X[0];
    double result=0.0;
    if (x>1.0/3.0 &&x<2.0/3.0 &&y>1.0/3.0 &&y<2.0/3.0 )
    {
        // Apply the first function for y > 0.5
        result = x>0.5 ? 1+1*Math.Sin(100*x)*Math.Sin(100*y): 1+1*Math.Cos(100*x)*Math.Cos(100*y);
    }
    else 
    {
        // Apply the second function for y < 0.75
        result = x+y;
    }
    
    return result;
};
Func<double[],double> discCircB= delegate(double[] X){
    var y=X[1];
    var x=X[0];
    double result=0.0;
    if (x>1.0/3.0 &&x<2.0/3.0 &&y>1.0/3.0 &&y<2.0/3.0 )
    {
        // Apply the first function for y > 0.5
        result = x< 0.5 ? 4+ 1*Math.Sin(100*x)*Math.Sin(100*y): 4+1*Math.Cos(100*x)*Math.Cos(100*y);
    }
    else 
    {
        // Apply the second function for y < 0.75
        result = 4;
    }
    
    return result;
};

In [64]:
BoSSS.Solution.Application.DeleteOldPlotFiles();

pSAIDT.ConservativeFields[0].GetSpeciesShadowField("L").ProjectField(discCircA);
pSAIDT.ConservativeFields[0].GetSpeciesShadowField("R").ProjectField(discCircB);
pSAIDT.LevelSetOpti.ProjectFromFunction(discCirc);
pSAIDT.LevelSetOpti.ProjectOntoLevelSet(pSAIDT.LsTBO);
plotSF("DGLevelSet");

rm DGLevelSet.plt;


## Spline level set

In [8]:
BoSSS.Solution.Application.InitMPI();
var pSAIDT = new SAIDTMain();
var CSAIDT = SAIDTHardCodedControl.StraightShock(
                    dbPath: dbPath,
                    MaxIterations: 100,
                    dgDegree: 0,
                    numOfCellsX: 10,
                    numOfCellsY: 10,
                    agg: 0.4,
                    ImmediatePlotPeriod: -1,
                    optiLevelSetType: OptiLevelSetType.SplineLevelSet,
                    LSDegree: 3,
                    isFarConfig:true
                    );
CSAIDT.SessionName="SAIDT_SplineLevelSet";
CSAIDT.ProjectName="SAIDT_SplineLevelSet";
pSAIDT.Init(CSAIDT);
pSAIDT.InitializeEverything();


## SpecFemField

In [80]:
using SAIDT;
BoSSS.Solution.Application.InitMPI();
var pSAIDT = new SAIDTMain();
var CSAIDT = SAIDTHardCodedControl.StraightShock(
                    dbPath: dbPath,
                    MaxIterations: 50,
                    dgDegree: 0,
                    numOfCellsX: 4,
                    numOfCellsY: 4,
                    OptiNumOfCellsX: 4,
                    OptiNumOfCellsY: 4,
                    agg: 0.4,
                    ImmediatePlotPeriod: 5,
                    optiLevelSetType: OptiLevelSetType.SpecFemField,
                    LSDegree: 3,
                    isFarConfig:true
                    );
CSAIDT.SessionName="SAIDT_CGLevelSet";
CSAIDT.ProjectName="SAIDT_CGLevelSet";
pSAIDT.Init(CSAIDT);

pSAIDT.InitializeEverything();

Session ID: 00000000-0000-0000-0000-000000000000, DB path: 'EMPTY'
Session directory 'EMPTY\sessions\00000000-0000-0000-0000-000000000000'.
Grid repartitioning method: METIS
Grid repartitioning options: 
Number of cell Weights: 0


*****************************************************************************************
*  X   X   DDD   GGG   -   III  SSS  TTTTT  *
*   X X    D  D G      -    I  S      T     *
*    X     D  D G  GG  -    I   SSS   T     *      XDG Implicit Shock Tracking Solver    
*   X X    D  D G   G  -    I      S  T     *
*  X   X   DDD   GGG   -   III  SSS   T     *
*****************************************************************************************
*  
*  solver:                             SAIDT.SAIDTMain
*  project:                            SAIDT_CGLevelSet
*  session:                            SAIDT_CGLevelSet
*  optimization problem:               FullEnRes
*  merit function:                     L1Merit
*  level set type:                     SpecFemF

In [90]:
 public static int findMaximumGreatness(List<int> arr)
    {
        var sorted_arr = arr;
        sorted_arr.Sort(); // sorted ascending
        Console.WriteLine(sorted_arr);
        List<int> rearranged_arr  = sorted_arr.Select(item => item).ToList();
        var tmp=rearranged_arr.First();
        rearranged_arr.RemoveAt(0);
        rearranged_arr.Add(tmp);
        
        int cnt=0;
        for(int i=1;i<arr.Count;i++){
            if(rearranged_arr[i]>sorted_arr[i]){
                cnt++;
            }
        }
        return cnt;
        
    }
List<int>sorted_arr= new List<int> {1,3,5,2,1,3,1};
sorted_arr.Sort();
sorted_arr

[ 1, 1, 1, 2, 3, 3, 5 ]

In [91]:
List<int> rearranged_arr  = sorted_arr.Select(item => item).ToList();
var tmp=rearranged_arr.First();
rearranged_arr.RemoveAt(0);
rearranged_arr.Add(tmp);
rearranged_arr

[ 1, 1, 2, 3, 3, 5, 1 ]

In [92]:
 int cnt=0;

In [94]:

        int n= sorted_arr.Count;
var i=n-1;
if(rearranged_arr[i]>sorted_arr[i]){
    cnt++;
    rearranged_arr.RemoveAt(rearranged_arr.Count-1); //remove last
    sorted_arr.RemoveAt(sorted_arr.Count-1);
}


In [97]:
                rearranged_arr.RemoveAt(0);
                sorted_arr.RemoveAt(sorted_arr.Count-1);

In [98]:
rearranged_arr

[ 1, 2, 3, 3, 5, 1 ]

In [99]:
sorted_arr

[ 1, 1, 1, 2, 3, 3 ]

In [81]:
BoSSS.Solution.Application.DeleteOldPlotFiles();
pSAIDT.ConservativeFields[0].GetSpeciesShadowField("L").ProjectField(discCircA);
pSAIDT.ConservativeFields[0].GetSpeciesShadowField("R").ProjectField(discCircB);
pSAIDT.LevelSetOpti.ProjectFromFunction(discCirc);
pSAIDT.LevelSetOpti.ProjectOntoLevelSet(pSAIDT.LsTBO);
public void plotSF(string name){
    List<DGField> flds= new List<DGField>();
foreach(var field in pSAIDT.RegisteredFields){
    if(field is XDGField xfield){
        flds.Add(xfield.GetSpeciesShadowField("L"));
        flds.Add(xfield.GetSpeciesShadowField("R"));
        flds.Add(field);
    }else{
        flds.Add(field);
    }
}

var texplot = new Tecplot(flds[0].GridDat, 2);
texplot.PlotFields( name,0.0,flds);
}
plotSF("CGLevelSet");

rm CGLevelSet.plt SAIDT_CGLevelSet_0.plt;


In [13]:
var si= database.Sessions.Pick(0);
pSAIDT.PlotShadowFields(si);

In [14]:
using System.IO;
static void AppendRowToCSV(string filePath, double[] rowData)
    {
        // Create a StreamWriter in append mode to append rows to the CSV file
        using (StreamWriter writer = new StreamWriter(filePath, true))
        {
            int cols = rowData.Length;

            // Write double values of the new row to the file
            for (int i = 0; i < cols; i++)
            {
                writer.Write(rowData[i]);

                // Add a comma after each value except for the last one in a row
                if (i < cols - 1)
                {
                    writer.Write(";");
                }
            }

            // Move to the next line after writing the row
            writer.WriteLine();
        }
    }
    static void WriteCSV(string filePath, string[] data)
    {
        // Create a StreamWriter to write to the CSV file
        using (StreamWriter writer = new StreamWriter(filePath))
        {
            int cols = data.Length;

            // Loop through the data array and write to the file
                for (int j = 0; j < cols; j++)
                {
                    // Write each cell value to the file
                    writer.Write(data[ j]);

                    // Add a comma after each cell value except for the last one in a row
                    if (j < cols - 1)
                    {
                        writer.Write(";");
                    }
                }

                // Move to the next line after each row
                writer.WriteLine();
            
        }
    }
    static void DeleteCSVFiles()
    {
        try
        {
            // Get all CSV files in the specified directory
            string[] csvFiles = Directory.GetFiles("*.csv");

            // Delete each CSV file
            foreach (string file in csvFiles)
            {
                File.Delete(file);
                Console.WriteLine($"Deleted: {file}");
            }
        }
        catch (Exception ex)
        {
            Console.WriteLine($"Error deleting CSV files: {ex.Message}");
        }
    }

## Measure wall-clock time for computation of sensitivities and residuals 

In [15]:
using System.Diagnostics;
BoSSS.Solution.Application.InitMPI();
DeleteCSVFiles();

int nRefs=4;
var optLsTypes=new OptiLevelSetType[] {OptiLevelSetType.SinglePhaseField,OptiLevelSetType.SplineLevelSet,OptiLevelSetType.SpecFemField};
var CellsVsRuntime_dRdPhi = MultidimensionalArray.Create(nRefs,optLsTypes.Length +1);
var CellsVsRuntime_Residuals = MultidimensionalArray.Create(nRefs,optLsTypes.Length +1);
var CellsVsRuntime_dRdU = MultidimensionalArray.Create(nRefs,optLsTypes.Length +1);
string[] stringArray = new string[] { "nCells" }.Concat(optLsTypes.Select(x => x.ToString())).ToArray();

//Create files for results
string filePathdRdPhi = "TimeDRdPhi.csv";
string filePathdRdU = "TimedRdU.csv";
string filePathResiduals = "TimeResiduals.csv";
string filePathDOFs = "DOFs.csv";
WriteCSV(filePathdRdPhi,stringArray);
WriteCSV(filePathdRdU,stringArray);
WriteCSV(filePathResiduals,stringArray);
WriteCSV(filePathDOFs,stringArray);

for(int n =0; n< nRefs;n++){
    //counter for level set type
    int iLS=1;
    //save cell resolution
    int reso= (int) Math.Pow(2,n+1);
    CellsVsRuntime_dRdPhi[n,0]=reso*reso;
    double[] dRdPhiResults= new double[optLsTypes.Length +1];
    dRdPhiResults[0]=reso*reso;
    double[] dRdUResults=dRdPhiResults.CloneAs();
    double[] resiudalResults=dRdPhiResults.CloneAs();
    double[] DOFs=dRdPhiResults.CloneAs();
    foreach(OptiLevelSetType lstype in optLsTypes){
        var pSAIDT = new SAIDTMain();
        var CSAIDT = SAIDTHardCodedControl.StraightShock(
                            dbPath: null,
                            MaxIterations: 1,
                            dgDegree: 0,
                            numOfCellsX: reso,
                            numOfCellsY: reso,
                            OptiNumOfCellsX: reso,
                            OptiNumOfCellsY: reso,
                            agg: 0.4,
                            ImmediatePlotPeriod: -1,
                            optiLevelSetType: lstype,
                            LSDegree: 3,
                            isFarConfig:true);

        pSAIDT.Init(CSAIDT);    
        pSAIDT.InitializeEverything();
        //pSAIDT.RunSolverMode();

        //Measure time for creation of Jacobians dRdPhi
        Stopwatch stopwatch = new Stopwatch();stopwatch.Start();
        pSAIDT.FD_LevelSet();
        stopwatch.Stop();
        CellsVsRuntime_dRdPhi[n,iLS] =stopwatch.ElapsedMilliseconds;
        dRdPhiResults[iLS]=stopwatch.ElapsedMilliseconds;

        //Measure time for creation of Jacobians dRdU
        stopwatch = new Stopwatch();stopwatch.Start();
        pSAIDT.Oproblem.GetJacobians(pSAIDT.ConservativeFields, pSAIDT.XSpatialOperator.LinearizationHint);
        stopwatch.Stop(); 
        CellsVsRuntime_dRdU[n,iLS] =stopwatch.ElapsedMilliseconds;
        dRdUResults[iLS]=stopwatch.ElapsedMilliseconds;

        //Measure time for evaluation of the Residuals
        stopwatch = new Stopwatch();stopwatch.Start();
        pSAIDT.ComputeResiduals();
        stopwatch.Stop(); Console.WriteLine("Elapsed Time: " + stopwatch.ElapsedMilliseconds + "ms");
        CellsVsRuntime_Residuals[n,iLS] =stopwatch.ElapsedMilliseconds;
        resiudalResults[iLS]=stopwatch.ElapsedMilliseconds;

        DOFs[iLS]=pSAIDT.LevelSetOpti.GetLength();

        iLS++;
    }
    // Append the new row to the existing CSV file
    AppendRowToCSV(filePathdRdPhi, dRdPhiResults);
    AppendRowToCSV(filePathdRdU, dRdUResults);
    AppendRowToCSV(filePathResiduals, resiudalResults);
    AppendRowToCSV(filePathDOFs, DOFs);
}

In [16]:
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

In [17]:
CellsVsRuntime_dRdPhi

Dimension,2
Lengths,"[ 4, 4 ]"
Storage,"[ 4, 1087, 174, 911, 16, 5625, 388, 3309, 64, 29468, 1001, 19166, 256, 242434, 3516, 166633 ]"
IsContinuous,True
StructureType,General
Length,16
NoOfCols,4
NoOfRows,4
IsLocked,False


In [18]:
var plot = new Plot2Ddata();
int count=0;
for(int iLS=0;iLS<CellsVsRuntime_dRdPhi.Lengths[1]-1; iLS ++){
    plot.AddDataGroup(optLsTypes[iLS].ToString(),CellsVsRuntime_dRdPhi.ExtractSubArrayShallow(-1,0).To1DArray(),CellsVsRuntime_dRdPhi.ExtractSubArrayShallow(-1,iLS+1).To1DArray(), GetFormat(count));
    count++;
}
plot.Xlabel="number of Cells";
plot.LogY = true;
plot.LogX = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:600)

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 2 
 
 
 
 
 10 3 
 
 
 
 
 10 4 
 
 
 
 
 10 5 
 
 
 
 
 10 6 
 
 
 
 
 10 0 
 
 
 
 
 10 1 
 
 
 
 
 10 2 
 
 
 
 
 10 3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 number of Cells 
 
 
 
 
 SinglePhaseField 
 
 
 SinglePhaseField 
 
 
 
 
 
 
 
 
 
 
 SplineLevelSet 
 
 
 SplineLevelSet 
 
 
 
 
 
 
 
 
 
 
 SpecFemField 
 
 
 SpecFemField